In [47]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from App.helper.functions import station_upper
import App.helper.config as config

In [48]:
# Read twitter data
daten = pd.read_csv('App/Daten/data_expanded/combined_data_twitter_long.csv')
daten['Einschr_type'].fillna('Keine', inplace=True)
print(len(daten))
daten.head(2)

423227


,BETRIEBSTAG,LINIEN_ID,LINIEN_TEXT,haltestelle_an,AN_soll,AN_ist,AN_diff,haltestelle_ab,AB_soll,AB_ist,...,weekday,ab_hour,ab_minute,disturbance_overlap,Einschr_type,feiertag,Temperatur,Niederschlag,Luftfeuchtigkeit,Wind
0,2023-03-25,843,IC8,Zürich HB,2023-03-26 01:23:00,2023-03-26 03:21:39,118.65,Olten,2023-03-26 01:47:00,2023-03-26 01:47:36,...,5,1,47,False,Keine,0,11.0,8.0,84.6,7.5
1,2023-03-25,843,IC8,Zürich HB,2023-03-26 01:23:00,2023-03-26 03:21:39,118.65,Bern,2023-03-26 01:06:00,2023-03-26 01:07:39,...,5,1,6,False,Keine,0,1.2,4.8,75.3,6.8


In [49]:
daten.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423227 entries, 0 to 423226
Data columns (total 21 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   BETRIEBSTAG          423227 non-null  object 
 1   LINIEN_ID            423227 non-null  int64  
 2   LINIEN_TEXT          423227 non-null  object 
 3   haltestelle_an       423227 non-null  object 
 4   AN_soll              423227 non-null  object 
 5   AN_ist               423227 non-null  object 
 6   AN_diff              423227 non-null  float64
 7   haltestelle_ab       423227 non-null  object 
 8   AB_soll              423227 non-null  object 
 9   AB_ist               423227 non-null  object 
 10  AB_diff              423227 non-null  float64
 11  weekday              423227 non-null  int64  
 12  ab_hour              423227 non-null  int64  
 13  ab_minute            423227 non-null  int64  
 14  disturbance_overlap  423227 non-null  bool   
 15  Einschr_type     

In [50]:
daten['Linien_typ'] = daten['LINIEN_TEXT'].str.replace('\d+', '', regex=True)

In [51]:
daten['Linien_typ'].value_counts()

IC     228845
IR     146111
EC      34058
ICE     14213
Name: Linien_typ, dtype: int64

In [52]:
daten.isna().sum().sort_values(ascending=False).head(1)

BETRIEBSTAG    0
dtype: int64

In [53]:
# replace True and False with 1 and 0 in the column 'disturbance_overlap'
daten['disturbance_overlap'] = daten['disturbance_overlap'].replace(True, 1)
daten['disturbance_overlap'] = daten['disturbance_overlap'].replace(False, 0)

In [54]:
daten.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423227 entries, 0 to 423226
Data columns (total 22 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   BETRIEBSTAG          423227 non-null  object 
 1   LINIEN_ID            423227 non-null  int64  
 2   LINIEN_TEXT          423227 non-null  object 
 3   haltestelle_an       423227 non-null  object 
 4   AN_soll              423227 non-null  object 
 5   AN_ist               423227 non-null  object 
 6   AN_diff              423227 non-null  float64
 7   haltestelle_ab       423227 non-null  object 
 8   AB_soll              423227 non-null  object 
 9   AB_ist               423227 non-null  object 
 10  AB_diff              423227 non-null  float64
 11  weekday              423227 non-null  int64  
 12  ab_hour              423227 non-null  int64  
 13  ab_minute            423227 non-null  int64  
 14  disturbance_overlap  423227 non-null  int64  
 15  Einschr_type     

In [55]:
data = daten.drop(['AN_soll', 'AN_ist', 'AB_soll', 'AB_ist', 'AB_diff', 'LINIEN_ID', 'LINIEN_TEXT', 'disturbance_overlap'], axis=1)
data = pd.get_dummies(data, columns=['haltestelle_ab', 'haltestelle_an', 'Linien_typ', 'Einschr_type'])

In [56]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423227 entries, 0 to 423226
Data columns (total 33 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   BETRIEBSTAG                  423227 non-null  object 
 1   AN_diff                      423227 non-null  float64
 2   weekday                      423227 non-null  int64  
 3   ab_hour                      423227 non-null  int64  
 4   ab_minute                    423227 non-null  int64  
 5   feiertag                     423227 non-null  int64  
 6   Temperatur                   423227 non-null  float64
 7   Niederschlag                 423227 non-null  float64
 8   Luftfeuchtigkeit             423227 non-null  float64
 9   Wind                         423227 non-null  float64
 10  haltestelle_ab_Basel SBB     423227 non-null  uint8  
 11  haltestelle_ab_Bern          423227 non-null  uint8  
 12  haltestelle_ab_Lugano        423227 non-null  uint8  
 13 

In [57]:
data.to_csv('App/Daten/data_new/less_wide.csv', index=False)

In [61]:
data_detail = daten.drop(['AN_soll', 'AN_ist', 'AB_soll', 'AB_ist', 'AB_diff', 'LINIEN_ID', 'Linien_typ', 'disturbance_overlap'], axis=1)
data_detail['week'] = pd.to_datetime(data_detail['BETRIEBSTAG']).dt.isocalendar().week
data_detail = pd.get_dummies(data_detail, columns=['haltestelle_ab', 'haltestelle_an', 'LINIEN_TEXT', 'Einschr_type'])

In [62]:
data_detail.to_csv('App/Daten/data_new/detail.csv', index=False)